In [1]:
import sklearn as sk

from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [2]:
import imdb_functions # Includes load_data()
import numpy as np

# The data is IMDB data, change the path to data directory.
X_train_corpus , y_train, X_test_corpus , y_test = imdb_functions.load_imdb(path = "../../Fall_19/aclImdb")

Loading the imdb data
Train Data loaded.
Test Data loaded.


In [3]:
# Try with the configuration below
# Decision Tree, min_df=100

token = r"(?u)\b[\w\'/]+\b"
vectorizer = CountVectorizer(token_pattern=token, 
                             binary=True,
                             ngram_range=(1,1),
                             min_df=100,
                             stop_words=["the","a","of","and","br","to"])
X_train_vector = vectorizer.fit_transform(X_train_corpus)
X_test_vector = vectorizer.transform(X_test_corpus)

In [4]:
X_train_vector.sum(axis=0)

matrix([[ 153,  219, 1418, ...,  320,  255,  123]], dtype=int64)

In [40]:
dt = DecisionTreeClassifier(max_depth=100)

In [41]:
dt.fit(X_train_vector, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [9]:
vectorizer.get_feature_names()

['0',
 '000',
 '1',
 '1/10',
 '1/2',
 '10',
 '10/10',
 '100',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '1930s',
 "1950's",
 '1950s',
 '1970s',
 '1980s',
 '1st',
 '2',
 '2/10',
 '20',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '20th',
 '24',
 '25',
 '3',
 '3/10',
 '30',
 '4',
 '4/10',
 '40',
 '45',
 '5',
 '50',
 "50's",
 '6',
 '60',
 "60's",
 '60s',
 '7',
 '7/10',
 "70's",
 '70s',
 '8',
 '8/10',
 '80',
 "80's",
 '80s',
 '9',
 '9/10',
 '90',
 "90's",
 '99',
 'abandoned',
 'abilities',
 'ability',
 'able',
 'about',
 'above',
 'absence',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'academy',
 'accent',
 'accents',
 'accept',
 'acceptable',
 'accepted',
 'accident',
 'accidentally',
 'accomplished',
 'according',
 'account',
 'accurate',
 'accused',
 'achieve',
 'achieved',
 'achievement',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'actions',
 'actor',
 'actors',
 'actress',
 'actresses',
 'acts',
 'actual',
 'actually',
 'ad',
 'adam',
 

In [33]:
len(dt.feature_importances_), len(dt.feature_importances_.nonzero()[0])

(3893, 1561)

In [22]:
# There is no negative value.
len([x for x in dt.feature_importances_ if x<0])

0

In [44]:
# Find biggest coefficients.
importances = list(zip(vectorizer.get_feature_names(), dt.feature_importances_))
rank_importances = sorted(importances, key = lambda x: x[1], reverse=True)
rank_importances[:150]

[('bad', 0.07514562206983294),
 ('worst', 0.04807761971027291),
 ('great', 0.030045300751230294),
 ('waste', 0.029893528901474873),
 ('awful', 0.020179242449785775),
 ('boring', 0.01446119934624131),
 ('excellent', 0.013934053127746366),
 ('best', 0.010608950353508461),
 ('no', 0.010022211134640052),
 ('wonderful', 0.009754170506901935),
 ('poor', 0.009091661920722268),
 ('nothing', 0.0066003224294136),
 ('terrible', 0.00597390542820861),
 ('stupid', 0.005783541158707123),
 ('perfect', 0.005323401848285497),
 ('love', 0.005167047114137656),
 ('poorly', 0.005101373477085532),
 ('worse', 0.004911341616396622),
 ('ridiculous', 0.004804857449792027),
 ('both', 0.003886967903438934),
 ('lame', 0.0035047811922774962),
 ('beautiful', 0.0033498947167068093),
 ('dull', 0.0031107462927367183),
 ('very', 0.003037325637035202),
 ('favorite', 0.0030353302010984345),
 ('disappointing', 0.002999107432519329),
 ('horrible', 0.002966517141940955),
 ('has', 0.0029168614808268993),
 ('acting', 0.00287161

In [45]:
feature_names = vectorizer.get_feature_names()

In [52]:
dt.feature_importances_[feature_names.index('bad')]

0.07514562206983294

In [59]:
freqs=np.sum(X_train_vector, axis=0).A1

In [60]:
freqs[feature_names.index('bad')]

5892

In [61]:
freqs[feature_names.index('worst')]

2264

In [63]:
freqs[feature_names.index('1/10')]

0.00632